In [1]:
import time
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [3]:
####### Section 1. Set up #######
torch.random.manual_seed(0)
model_id = "./models/Phi-3-medium-4k-instruct" # please replace with local model path
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
 
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [23]:
generation_args = {
    "max_new_tokens": 20,
    "return_full_text": False,
    "temperature": 0.0, #decoding할 때 확률값을 random으로 줌. 
    "do_sample": False,
    "batch_size": 100 , # batch size를 지정함. 
}

In [24]:
messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]
output = pipe(messages, **generation_args)
# print(output[0]['generated_text'])

In [ ]:
torch.cuda.empty_cache() #cache를 지움

In [7]:
data = load_dataset("json", data_files="./data/test_dataset.jsonl")['train']

In [8]:
def data_preprocessing(data):
    new_set = []
    for i in range(len(data)):
        temp = data[i]['message'][0]['content']
        new_set.append(temp)
    return new_set
data_list = data_preprocessing(data)


In [16]:
####### Section 3. Load data and Inference -> Performance evaluation part #######
start = time.time()
data = load_dataset("json", data_files="./data/test_dataset.jsonl")['train']
outs = pipe(KeyDataset(data, 'message'), **generation_args)
end = time.time()

/home/elicer/miniconda3/envs/hrenv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":838, please report a bug to PyTorch. 

In [ ]:
####### Section 4. Accuracy (Just for leasderboard) #######
print("===== Answers =====")
correct = 0
for i, out in enumerate(outs):
    correct_answer = data[i]["answer"]
    answer = out[0]["generated_text"].lstrip().replace("\n","")
    if answer == correct_answer:
        correct += 1
    # print(answer)
 
print("===== Perf result =====")
print("Elapsed_time: ", end-start)
print(f"Correctness: {correct}/{len(data)}")

===== Answers =====
===== Perf result =====
Elapsed_time:  58.36229968070984
Correctness: 24/30
